In [1]:
import pandas as pd
import numpy as np
import config
import datetime
import sys
from sqlalchemy import text
print(sys.version)
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)

3.12.7 (tags/v3.12.7:0b05ead, Oct  1 2024, 03:06:41) [MSC v.1941 64 bit (AMD64)]
3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [2]:
specialite=21
year=23
dico={}
year, specialite

(23, 21)

In [3]:
spe = pd.read_sql("select * from specialite", config.connection_string)
spe

,id,label,label_long,psp_spe_snds,psp_act_snds
0,1,Psychiatre,Psychiatre,33.0,NaN
1,2,Anest,Anesthésiste,2.0,NaN
2,3,Dermato,Dermatologue,5.0,NaN
3,4,Gastro,Gastro-entérologue,8.0,NaN
4,5,Gyneco,Gynécologue,7.0,NaN
5,6,Opthtalmo,Opthtalmogue,15.0,NaN
6,7,Pediatre,Pédiatre,12.0,NaN
7,8,Radio,Radiologue,6.0,NaN
8,10,Generaliste,Généraliste,1.0,NaN
9,11,Cardio,Cardiologue,3.0,NaN


In [4]:
def get_ps(year, specialite):
    sql=f"""
select sp.specialite_id, s.label_long as label, ds.annee as year, ps.id as p_id, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id
join date_source ds on ds.id=tds.date_source_id
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join specialite s on s.id=sp.specialite_id
where sp.specialite_id={specialite}
and ds.annee={year}
"""
    print(f"Quering PS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string).dropna()


In [5]:
def get_pa(year, specialite, is_medecin=True):
    if is_medecin:
        sql=f"""
select sp.specialite_id, ds.annee as year, pa.id as p_id, an.lon, an.lat
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on d.code_diplome=ps.code_diplome
join profession_diplome pd on pd.diplome_id=d.id
join profession p on p.id=pd.profession_id
join specialite_profession sp on sp.profession_id=p.id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
where sp.specialite_id={specialite}
and ps.code_mode_exercice='L'
and ds.annee={year}
and pands.adresse_norm_id is not null
"""
    else:
        sql=f"""
select sp.specialite_id, ds.annee as year, pa.id as p_id, an.lon, an.lat
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join profession_code_profession pcp on pcp.code_profession_id=ps.code_profession
join specialite_profession sp on sp.profession_id=pcp.profession_id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
where sp.specialite_id={specialite}
and ps.code_mode_exercice='L'
and ds.annee={year}
and ps.cp is not null
"""   
    print(f"Quering PA for year {year} and specialite {specialite} and is_medecin {is_medecin}")
    print(sql)
    return pd.read_sql(sql, config.connection_string)

In [6]:
def get_rpps(year, specialite):
    sql=f"""
select personne.id p_id,an.lon,an.lat from personne
join diplome_obtenu dio on dio.personne_id = personne.id
join diplome d on d.id=dio.diplome_id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join coord c on c.personne_id=personne.id
join adresse_norm an on c.adresse_norm_id=an.id
where sp.specialite_id={specialite}
and personne.date_maj >= '20{year}-01-01' and date_effet <= '20{year}-12-31'
"""
    print(f"Quering RPPS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string).dropna()

In [7]:
df2=get_pa(23,21, False)
# df2["p_id"].nunique()
len(df2)

Quering PA for year 23 and specialite 21 and is_medecin False

select sp.specialite_id, ds.annee as year, pa.id as p_id, an.lon, an.lat
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join profession_code_profession pcp on pcp.code_profession_id=ps.code_profession
join specialite_profession sp on sp.profession_id=pcp.profession_id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
where sp.specialite_id=21
and ps.code_mode_exercice='L'
and ds.annee=23
and ps.cp is not null



227383

In [8]:
nb_unique = len(df2.groupby(["p_id", "lon", "lat"]))
nb_unique

112274

In [9]:
nb = df2["p_id"].nunique()
nb

97903

In [10]:
ameli={1:6214,2:3859,3:2433,4:2021,5:4368,6:4295,7:2704,8:5518,9:36062,10:51488,11:4989,12:7777,13:842,14:115,15:1036,16:1971,17:1196,18:505,19:1449,20:704,21:99467,22:7600,23:75475,24:12717,25:2947,26:74219,27:37120}
ameli

{1: 6214,
 2: 3859,
 3: 2433,
 4: 2021,
 5: 4368,
 6: 4295,
 7: 2704,
 8: 5518,
 9: 36062,
 10: 51488,
 11: 4989,
 12: 7777,
 13: 842,
 14: 115,
 15: 1036,
 16: 1971,
 17: 1196,
 18: 505,
 19: 1449,
 20: 704,
 21: 99467,
 22: 7600,
 23: 75475,
 24: 12717,
 25: 2947,
 26: 74219,
 27: 37120}

In [11]:
columns=["specialite","label","year","ameli","nb_ps_cabinet","nb_ps","delta_ps","nb_pa_cabinet","nb_pa","delta_pa","nb_pa_ps_cabinet","nb_pa_ps","delta_pa_ps","nb_pa_outer_ps_cabinet","nb_pa_outer_ps","delta_pa_outer_ps","nb_rpps_cabinet","np_rpps","delta_rpps"]
columns=["specialite","label","year","ameli","nb_ps_cabinet","nb_ps","delta_ps","nb_pa_cabinet","nb_pa","delta_pa","nb_rpps_cabinet","np_rpps","delta_rpps"]
df = pd.DataFrame(columns=columns)
for specialite in range(21,28):
    for yy in range(20,26): 
        ps_df=get_ps(yy,specialite)
        label=ps_df["label"][0] if len(ps_df["label"])>0 else ""
        nb_unique_ps=len(ps_df.groupby(["p_id", "lon", "lat"]))
        nb_ps=ps_df["p_id"].nunique()
        pa_df=get_pa(yy,specialite,specialite<=20)
        nb_unique_pa=len(pa_df.groupby(["p_id", "lon", "lat"]))
        nb_pa=pa_df["p_id"].nunique()
        rpps_df=get_rpps(yy,specialite)
        nb_unique_rpps=len(rpps_df.groupby(["p_id", "lon", "lat"]))
        nb_rpps=rpps_df["p_id"].nunique()
        a=np.nan
        if yy==23 and specialite in ameli:
            a=ameli[specialite]
        df=pd.concat([pd.DataFrame([[specialite,label,yy,a,nb_unique_ps,nb_ps,(nb_ps-a)/a,nb_unique_pa,nb_pa,(nb_pa-a)/a,nb_unique_rpps,nb_rpps,(nb_rpps-a)/a]], columns=df.columns), df], ignore_index=True)
df=df.sort_values(by=['specialite',"year"])
df

Quering PS for year 20 and specialite 21
Quering PA for year 20 and specialite 21 and is_medecin False

select sp.specialite_id, ds.annee as year, pa.id as p_id, an.lon, an.lat
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join profession_code_profession pcp on pcp.code_profession_id=ps.code_profession
join specialite_profession sp on sp.profession_id=pcp.profession_id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
where sp.specialite_id=21
and ps.code_mode_exercice='L'
and ds.annee=20
and ps.cp is not null

Quering RPPS for year 20 and specialite 21
Quering PS for year 21 and specialite 21
Quering PA for year 21 and specialite 21 and is_medecin False

select sp.specialite_id, ds.annee as year, pa.id as p_id, an.lon, an.lat
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.in

,specialite,label,year,ameli,nb_ps_cabinet,nb_ps,delta_ps,nb_pa_cabinet,nb_pa,delta_pa,nb_rpps_cabinet,np_rpps,delta_rpps
41,21,,20,NaN,0,0,NaN,97234,94744,NaN,0,0,NaN
40,21,,21,NaN,0,0,NaN,105715,91710,NaN,0,0,NaN
39,21,,22,NaN,0,0,NaN,108245,94930,NaN,0,0,NaN
38,21,,23,99467,0,0,-1.000000,112274,97903,-0.015724,0,0,-1.0
37,21,,24,NaN,0,0,NaN,109231,97282,NaN,0,0,NaN
36,21,,25,NaN,0,0,NaN,114137,99550,NaN,0,0,NaN
35,22,Sage-femme,20,NaN,6487,6054,NaN,9080,6478,NaN,0,0,NaN
34,22,Sage-femme,21,NaN,7687,6802,NaN,9684,6981,NaN,0,0,NaN
33,22,Sage-femme,22,NaN,7508,6865,NaN,9808,7195,NaN,0,0,NaN
32,22,Sage-femme,23,7600,7500,6786,-0.107105,10463,7565,-0.004605,0,0,-1.0


In [12]:
df.to_excel("out/ps_stats.xlsx", index=False)
df.to_sql("ps_stats", config.connection_string, schema="apl", if_exists="replace", index=False)

42